In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config
from bcnf.train import Trainer
from bcnf import CondRealNVP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME = 'trajectory_FC'

In [3]:
config_path = os.path.join(get_dir("configs", "runs"), f"{MODEL_NAME}.yaml")
config = load_config(config_path)

In [4]:
model = CondRealNVP.from_config(config).to(device)

In [5]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/psaegert/Projects/bcnf/data/bcnf-data/test_resimulation_data_T2_dt_1_15.pkl...
Using trajectory data for training. Shapes:
X shape: torch.Size([2000, 30, 3])
y shape: torch.Size([2000, 21])


In [7]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: psaegert (balisticcnf). Use `wandb login --relogin` to force relogin


Train: -3.4786 - Val: -5.3454 (avg: -4.6714, min: -4.5698) | lr: 2.00e-04 - Patience: 2/500 - z: (0.0130 ± 0.1016) ± (0.8022 ± 0.1404):   1%|          | 465/50000 [03:36<6:24:54,  2.14it/s]       
Traceback (most recent call last):
  File "/home/psaegert/Projects/bcnf/src/bcnf/train/trainer.py", line 100, in train
    model = self._train(
            ^^^^^^^^^^^^
  File "/home/psaegert/Projects/bcnf/src/bcnf/train/trainer.py", line 227, in _train
    loss = self._train_batch(x, y, model, optimizer, loss_function)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/psaegert/Projects/bcnf/src/bcnf/train/trainer.py", line 306, in _train_batch
    loss.backward()
  File "/home/psaegert/miniconda3/envs/bcnf/lib/python3.11/site-packages/torch/_tensor.py", line 522, in backward
    torch.autograd.backward(
  File "/home/psaegert/miniconda3/envs/bcnf/lib/python3.11/site-packages/torch/autograd/__init__.py", line 266, in backward
    Variable._execution_engine.run_

distance_to_last_best_val_loss_fold_-1,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▄▄▂▁▂▂▂▁▂▁▂▂▁▄▇▅▂▄▁█▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr_fold_-1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time_fold_-1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████████████████████
train_loss_fold_-1,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_fold_-1,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
distance_to_last_best_val_loss_fold_-1,2
epoch,465
lr_fold_-1,0.0002
time_fold_-1,1710607034.14257
train_loss_fold_-1,-3.47858


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path}, f)